<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1">Introduction</a></span></li><li><span><a href="#Expected-shortfall" data-toc-modified-id="Expected-shortfall-2">Expected shortfall</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-3">Imports</a></span></li><li><span><a href="#Loading-the-data" data-toc-modified-id="Loading-the-data-4">Loading the data</a></span></li><li><span><a href="#Modelling" data-toc-modified-id="Modelling-5">Modelling</a></span></li><li><span><a href="#References" data-toc-modified-id="References-6">References</a></span></li></ul></div>

# Introduction
<hr style = "border:2px solid black" ></hr>

<div class="alert alert-warning">
<font color=black>

**What?** Expected shortfall

</font>
</div>

# Expected shortfall
<hr style = "border:2px solid black" ></hr>

<div class="alert alert-info">
<font color=bla ck>

- Unlike VaR (Value at risk), ES (Expected Shortfall) focuses on the tail of the distribution. More specifically, ES enables us to take into account unexpected risks in the market. However, this doesn’t mean that ES and VaR are two entirely different concepts. Rather, they are related—that is, it is possible to express ES using VaR.

</font>
</div>

# Imports
<hr style = "border:2px solid black" ></hr>

In [1]:
import pandas_datareader as pdr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import yfinance as yf
from scipy.stats import norm
import requests
from io import StringIO
import seaborn as sns; sns.set()
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

# Loading the data
<hr style = "border:2px solid black" ></hr>

In [2]:
def getDailyData(symbol, start, end):
    pd = pdr.get_data_yahoo(symbol, start, end)
    return pd

In [3]:
getDailyData(["IBM", "MSFT", "INTC"], '2020-01-01', '2020-12-21')

Attributes   Adj Close                              Close              \
Symbols            IBM        MSFT       INTC         IBM        MSFT   
Date                                                                    
2020-01-02  112.916939  156.592010  56.406250  129.464630  160.619995   
2020-01-03  112.016403  154.642151  55.720169  128.432129  158.619995   
2020-01-06  111.816284  155.041916  55.562557  128.202682  159.029999   
2020-01-07  111.891319  153.628220  54.635437  128.288712  157.580002   
2020-01-08  112.825218  156.075287  54.672523  129.359467  160.089996   
...                ...         ...        ...         ...         ...   
2020-12-15  110.510712  210.971619  48.145493  120.391968  214.130005   
2020-12-16  110.177246  216.045654  48.573078  120.028679  219.279999   
2020-12-17  110.177246  216.183594  48.126495  120.028679  219.419998   
2020-12-18  110.440521  215.365829  45.095421  120.315491  218.589996   
2020-12-21  108.281723  219.306839  44.050240  117.963669  222.589996   

Attributes                   High                                Low  \
Symbols          INTC         IBM        MSFT       INTC         IBM   
Date                                                                   
2020-01-02  60.840000  129.942642  160.729996  60.970001  128.843216   
2020-01-03  60.099998  128.929260  159.949997  60.700001  127.686424   
2020-01-06  59.930000  128.336517  159.100006  60.200001  127.342255   
2020-01-07  58.930000  129.024857  159.669998  59.799999  127.533463   
2020-01-08  58.970001  129.885284  160.800003  59.320000  128.030594   
...               ...         ...         ...        ...         ...   
2020-12-15  50.669998  120.391968  215.419998  51.009998  118.011475   
2020-12-16  51.119999  121.003822  220.110001  51.459999  119.780113   
2020-12-17  50.650002  120.544930  220.889999  51.320000  119.416824   
2020-12-18  47.459999  120.841301  219.690002  50.680000  119.474190   
2020-12-21  46.360001  118.718925  224.000000  46.450001  116.367111   

Attributes                               Open                         \
Symbols           MSFT       INTC         IBM        MSFT       INTC   
Date                                                                   
2020-01-02  158.330002  60.220001  129.063095  158.779999  60.240002   
2020-01-03  158.059998  59.810001  127.695984  158.320007  59.810001   
2020-01-06  156.509995  59.330002  127.552582  157.080002  59.590000   
2020-01-07  157.320007  58.889999  127.810707  159.320007  59.779999   
2020-01-08  157.949997  58.520000  128.594650  158.929993  58.889999   
...                ...        ...         ...         ...        ...   
2020-12-15  212.240005  50.209999  118.919693  215.169998  50.980000   
2020-12-16  214.720001  50.680000  120.391968  214.750000  50.709999   
2020-12-17  217.919998  50.220001  120.535370  219.869995  51.150002   
2020-12-18  216.020004  47.110001  120.066925  218.589996  50.570000   
2020-12-21  217.279999  45.240002  118.518166  217.550003  46.320000   

Attributes     Volume                           
Symbols           IBM        MSFT         INTC  
Date                                            
2020-01-02  3293436.0  22622100.0   18056000.0  
2020-01-03  2482890.0  21116200.0   15293900.0  
2020-01-06  2537073.0  20813700.0   17755200.0  
2020-01-07  3232977.0  21634100.0   21876100.0  
2020-01-08  4545916.0  27746500.0   23133500.0  
...               ...         ...          ...  
2020-12-15  4556585.0  27000600.0   25465600.0  
2020-12-16  4738485.0  35023300.0   28351000.0  
2020-12-17  3962248.0  32515800.0   26399000.0  
2020-12-18  7900229.0  63354900.0  119298400.0  
2020-12-21  6397022.0  37181900.0   53775700.0  

[246 rows x 18 columns]

In [4]:
stocks = getDailyData(["IBM", "MSFT", "INTC"], '2020-01-01', '2020-12-31')["Close"]

In [5]:
# Calculating logarithmic return; normalising data so they are comparable
stocks_returns = (np.log(stocks) - np.log(stocks.shift(1))).dropna()

In [6]:
stocks

Symbols,IBM,MSFT,INTC
Date,,,
2020-01-02,129.464630,160.619995,60.840000
2020-01-03,128.432129,158.619995,60.099998
2020-01-06,128.202682,159.029999,59.930000
2020-01-07,128.288712,157.580002,58.930000
2020-01-08,129.359467,160.089996,58.970001
...,...,...,...
2020-12-24,119.206497,222.750000,47.070000
2020-12-28,119.330788,224.960007,47.070000
2020-12-29,118.355644,224.149994,49.389999


In [7]:
stocks_returns

Symbols,IBM,MSFT,INTC
Date,,,
2020-01-03,-0.008007,-0.012530,-0.012238
2020-01-06,-0.001788,0.002581,-0.002833
2020-01-07,0.000671,-0.009160,-0.016827
2020-01-08,0.008312,0.015803,0.000679
2020-01-09,0.010513,0.012416,0.005580
...,...,...,...
2020-12-24,0.006356,0.007797,0.010679
2020-12-28,0.001042,0.009873,0.000000
2020-12-29,-0.008205,-0.003607,0.048112


# Modelling
<hr style = "border:2px solid black" ></hr>

In [8]:
stocks_returns_mean = stocks_returns.mean()
# Drawing random numbers for weights
weights  = np.random.random(len(stocks_returns.columns))
# Generating weights
weights /= np.sum(weights)
# Calculating covariance matrix
cov_var = stocks_returns.cov()
# Finding the portfolio standard deviation
port_std = np.sqrt(weights.T.dot(cov_var).dot(weights))

In [9]:
weights

array([0.43101341, 0.41792623, 0.15106036])

In [10]:
initial_investment = 1e6
conf_level = 0.95

In [11]:
def ES_parametric(initial_investment, conf_level):

    alpha = - norm.ppf(1 - conf_level, stocks_returns_mean, port_std) 
    for i, j in zip(stocks.columns, range(len(stocks.columns))):
        VaR_param = (initial_investment * alpha)[j]
        ES_param = (1 / (1 - conf_level)) \
            * initial_investment \
            * norm.expect(lambda x: x,
                          lb=norm.ppf(conf_level,
                                      stocks_returns_mean[j],
                                      port_std),
                          loc=stocks_returns_mean[j],
                          scale=port_std) 
        print(f"Parametric ES result for {i} is {ES_param}")

In [12]:
ES_parametric(initial_investment, conf_level)

Parametric ES result for IBM is 50793.18872278611
Parametric ES result for MSFT is 52374.84817267527
Parametric ES result for INTC is 50290.09758541978


<div class="alert alert-info">
<font color=bla ck>

- ES can also be computed based on the historical observations. 
- Like the historical simulation VaR method, parametric assumption can be relaxed.
- To do that, the first return (or loss) corresponding to the 95% is found, and then the mean of the observations greater than the 95% gives us the result.

</font>
</div>

In [13]:
def VaR_historical(initial_investment, conf_level):
    Hist_percentile95 = []
    for i, j in zip(stocks_returns.columns,
                    range(len(stocks_returns.columns))):
        Hist_percentile95.append(np.percentile(stocks_returns.loc[:, i],
                                               5))
        print("Based on historical values 95% of {}'s return is {:.4f}"
              .format(i, Hist_percentile95[j]))
        VaR_historical = (initial_investment - initial_investment *
                          (1 + Hist_percentile95[j]))
        print("Historical VaR result for {} is {:.2f} "
              .format(i, VaR_historical))
        print('--' * 35)

In [14]:
VaR_historical(initial_investment, conf_level)

Based on historical values 95% of IBM's return is -0.0371
Historical VaR result for IBM is 37081.56 
----------------------------------------------------------------------
Based on historical values 95% of MSFT's return is -0.0426
Historical VaR result for MSFT is 42583.69 
----------------------------------------------------------------------
Based on historical values 95% of INTC's return is -0.0425
Historical VaR result for INTC is 42485.35 
----------------------------------------------------------------------


# References
<hr style = "border:2px solid black" ></hr>

<div class="alert alert-warning">
<font color=black>

- https://github.com/abdullahkarasan/mlfrm/blob/main/codes/chp_5.ipynb
- Machine Learning for Financial Risk Management with Python Abdullah Karasan

</font>
</div>